In [3]:
import numpy as np
import pandas as pd
from prophet import Prophet
import time

In [20]:
nInst = 50
currentPos = np.zeros(nInst)

training_data = pd.read_csv(r'data/prices0to250.csv')
prediction = training_data.copy()

# Simulated date for prophet model
start_date = '2023-07-01'
date_range = pd.date_range(start=start_date, periods=250, freq='D')
prediction['ds'] = date_range

In [ ]:
def getPositionWithProphet(prcSoFar):
    '''
        [WORK IN PROGRESS]
        An attempt at a trading algorithm using the Prophet library.
        1. Grab existing data
        2. Iterate through each stock and fit the model
        3. Predict, number of days ahead proportional to the number of days of data 
        4. Use the prediction to determine the position
    '''
    global currentPos, training_data, prediction
    # Parameters for prophet
    ndays = 25 # Number of days to predict, stay at around 10%

    
    # Parameters for the trading algorithm
    

    
    # Add one row to the prediction dataframe, with date


    (nins, nt) = prcSoFar.shape # nins is the number of instruments and nt is the number of days of data
    for i in range(nins):
        # Create a dataframe with the date and price

        # Fit the model

        # Add predictions to the prediction dataframe
        ...
    # Begin determining positionn
    for i in range(nins):

    return currentPos